- 문서에서 토큰을 추출하기 위해선 str.split()과 같거나 그 이상의 기능을 수행하는 조작 메서드가 필요.
    - *we'll 과 같은 축약형을 we, will로 분리할 매커니즘이 필요.*
- 토큰을 모두 식별했다면 **어간 추출(stemming)**이 필요. **정규 표현식을 이용**해서 가능함.
- 다음으로 이 **토큰들을 이용해 단어 모음(bag of words)이라 부르는 문서의 벡터 표현을 생성.**
- ***단어 또는 토큰이 무엇을 표현할까?*** 문맥과 상황에 따라 다를 것.
    - don't를 하나의 의미 전달 단위로 볼 수도 있고, 2개의 단위로 볼 수도 있음.
    - ice cream같은 경우는 'ice cream'을 하나로 봐야할까, 'ice' + 'cream'으로 봐야 할까?
- 텍스트를 단어로 분할하기 위해선 어떤 매커니즘이 필요할까?
    - 한 단어로 구성된 **'Don't!'**를 보자.
    - 컴퓨터 측면에서 보면 이는 **단순히 1개의 단어**
    - 하지만 사람 측면에서 보면 **어떤 생략된 단어가 있음**을 알 수 있음.
        - 'Don't you dare!' 아니면 'Don't you do that!'을 줄인 것 일 것.
        - 이런 축약형 단어는 추후 chapter.4에서 살펴보자.
        - 지금은 우선 토큰들을 주어진대로 식별하는 토큰 생성기를 구축해보자.
- **n-gram** : ***문자열에서 추출된 n쌍의 토큰(단어벡터).***
    - 이를 이용하면 'ice cream'같은 단어를 'ice', 'cream', 'ice cream'으로 추출할 수 있음.
        - **벡터 표현이 1개가 아닌 2개(더 나아가 n개)로 이뤄지는 것.**
- 텍스트를 수치 벡터로 변환하는 것은 손실이 큰 추출과정.
    - But, 텍스트에 담긴 정보를 통해 ML모형을 구축할 수 있음. 어느정도의 정보는 유지하는 셈.

## Stemming Overview  <br>

- **어간 추출(stemming)**은 **한 단어의 여러 변형을 동일한 '통' or '군집'으로 묶는 것**을 의미.
- ***복수형을 단수형으로 바꾸기 위해 s를 그냥 지워도 될까?***
    - words에서 s를 지워 word를 얻을 수 있음.
    - bus에서 s를 지우면 bu. 이 방법이 통하지 않음.
    - 어떠한 매커니즘이 필요.
- **한 단어 또는 단어의 한 부분에 있는 개별 글자가 그 단어의 의미에 관한 정보를 제공함.**
- **글자 하나 때문에 단어의 의미가 완전히 달라질 수도 있음.**
- 전통적인 어간 추출 방식을 사용해보자.

## Vocabulary building with Token Generator

<br>

- **토큰화(Tokenization)** : 문서 분할(segmentation)의 한 종류. **주어진 텍스트를 더 작고 구체적인 정보를 담은 조각으로 쪼개는 것.**
- 텍스트를 토큰으로 나누는 분할을 Tokenization 또는 Token Generation이라 함.
- NLP의 기본 구축요소들에 **컴파일러 구성 요소를 다음과 같이 대응**할 수 있음.
    - Token Generator == Scanner, Lexer
    - Vocabulary == Vocabulary Dictionary (어휘집)
    - Parser == Compiler
    - Token, n-gram, etc... == Token, 기호
- **토큰화는 NLP 파이프라인 첫 단계**인 만큼 파이프라인 나머지 부분에 큰 영향을 미침.
    - **구조가 없는 자료인 자연어 텍스트를 정보 조각들로 분할.**
    - 이런 **정보 조각들을 개별 요소로 취급해 개수를 셀 수 있음.**
    - 이 **횟수를 그대로 벡터 성분으로 이용**해 벡터를 만들면 **문서를 대표하는 하나의 벡터 표현**이 됨.
        - 이렇게 생성된 **단어모음 벡터의 가장 일반적인 용도는 문서 검색.**

<br>

- 토큰화의 가장 간단한 방법은 **문자열 안 공백(whitespace)을 구분자(delimiter)로 사용**하는 것. **str.split()**
    - **한국어** 같은 경우는 **단어를 좀 더 세부적인 단위인 형태소로 분리하는 것이 나을 수도** 있음.
    - **명사들만 추출하고 조사들은 불용어로 처리**하는 것이 바람직한 경우들. (-이, -을 제거)


               split을 이용한 간단한 토큰 분할을 보자.
   

In [1]:
sentence = 'Charlie started Data Science study at the age of 23.'
sentence.split()

['Charlie',
 'started',
 'Data',
 'Science',
 'study',
 'at',
 'the',
 'age',
 'of',
 '23.']

- 문장 끝 23. 에 마침표가 포함되어 있는것만 빼면 적당히 나눠졌음.
    - **마침표 같은 문장 부호는 의미 있는 토큰과 분리하는 것이 원칙.**
        - 26. 이라는 토큰은 컴퓨터 언어에서 부동소수점 수 26.0을 나타내는덴 완벽.
        - But, 다른 단어 26들과는 구분되는 다른 토큰이 되어버림.
        - 26. 26! 26? 들의 문장 부호를 제거해 모두 26이 되도록 하는것이 좋을 것.
    - 좀 더 정교하게 한다면 이런 **문장부호들을 별개의 토큰으로 분리**할 수 있을 것.
- 문장부호 제거는 추후 진행하고, **각 단어를 나타내는 수치 벡터 표현**을 만들어보자.
    - 이를 **one-hot vector**라고 함.
    - **원핫 벡터들의 시퀀스는 기존의 원문 텍스트를 온전히 표현**할 수 있음.
    - 이 과정을 통해 **주어진 텍스트가 수치적인 자료구조(2차원 수치배열)로 변환**됨.
        - NLP의 첫번째 문제인 단어들을 수치들로 변환하는 문제가 여기서 해결되는 셈.

In [2]:
import numpy as np

token_sequence = sentence.split()

# 고유 단어들만을 모아 어휘사전을 만듦.
vocab = sorted(set(token_sequence))
', '.join(vocab) # 토큰을 숫자>영대문자>영소문자 순으로 정렬.

'23., Charlie, Data, Science, age, at, of, started, study, the'

In [3]:
num_tokens = len(token_sequence)
vocab_size = len(vocab)

# 너비가 어휘의 고유 단어 개수, 높이가 문서의 길이(토큰개수)인 빈 테이블(2차원 배열) 생성.
# 현재 예시에선 10*10 크기의 테이블이 생성됨.
onehot_vectors = np.zeros((num_tokens, vocab_size), int)

# 문장의 각 단어에 대해, 어휘에서 그 단어에 해당하는 위치의 테이블 값을 1로 설정.
for i, word in enumerate(token_sequence):
    onehot_vectors[i, vocab.index(word)] = 1

print(' '.join(vocab))
onehot_vectors

23. Charlie Data Science age at of started study the


array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

- 이 0과 1들이 문장의 원핫 단어 벡터들. 근데 뭐가 뭔지 알아보기가 힘듦.
- 그러니 pd.DataFrame으로 그려서 살펴보자.

In [4]:
import pandas as pd

pd.DataFrame(onehot_vectors, columns=vocab)

,23.,Charlie,Data,Science,age,at,of,started,study,the
0,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0
2,0,0,1,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0
5,0,0,0,0,0,1,0,0,0,0
6,0,0,0,0,0,0,0,0,0,1
7,0,0,0,0,1,0,0,0,0,0
8,0,0,0,0,0,0,1,0,0,0
9,1,0,0,0,0,0,0,0,0,0


- 원핫 벡터는 모든 성분 중 단 하나만 0이 아닌 대단히 sparse한 벡터.
    - 문장이 커지면 메모리 낭비가 매~우 심해짐.
- 좀더 보기 편하려면 **0 $\rightarrow$ ' '(공백)**으로 바꿔서 봐보자. 파이프라인 넣을때 이런 짓 하면 안됩니다.

In [5]:
df = pd.DataFrame(onehot_vectors, columns=vocab)
df[df == 0] = ' '
df

,23.,Charlie,Data,Science,age,at,of,started,study,the
0,,1,,,,,,,,
1,,,,,,,,1,,
2,,,1,,,,,,,
3,,,,1,,,,,,
4,,,,,,,,,1,
5,,,,,,1,,,,
6,,,,,,,,,,1
7,,,,,1,,,,,
8,,,,,,,1,,,
9,1,,,,,,,,,


- 문장 1개만 있는 문서의 이런 표현에서, **각 row는 한 단어에 대한 벡터.**
- 현재 문장은 10개의 단어, 각 단어는 한번씩만 사용 됐음.
    - **한 col의 1은 해당 어휘 단어가 문서의 해당 위치에 출현한다는 의미.**
    - **문서의 세번째 단어가 뭔지 알고싶으면 테이블의 3번째 row에서 값이 1인 col**을 찾으면 됨.
        - 3번째 단어를 알고싶으니 **row index==2 에서 value가 1인 col을 찾으면 Data.**
        - 실제 **sentence를 보면 Data가 3번째에 출현**함.
        - NLP 파이프라인에서 문장의 한 단어 'Data'는 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]라는 벡터가 되는 셈.
- 단어의 벡터표현, 문서의 테이블 표현이 가진 **한 가지 장점은 어떤 정보도 소실되지 않는다는 것.**
    - **각 col이 어떤 단어에 대응되는지에 대한 정보만 유지하면 원핫 벡터 테이블을 통해 元 문서 복원이 가능.**
    - 이런 장점때문에 NN 같은데서 원핫 단어 벡터를 많이 사용함. 물론 메모리와 trade-off 해야하지만...
- 원핫 벡터 테이블을 **피아노 롤** or **음악상자 원통** 같은 개념으로 생각할 수 있음.
- 궁극적으로 하고자 하는것은 원핫 벡터들을 이용해 새로운 문장을 만드는것. LSTM을 이용한 NN 같은 모형을 사용해 만들어낼 수 있음. 추후 chapter 9, 10. 에서.

- 원핫 벡터 테이블은 컴퓨터가 좋아하는 **이진수**. 또한 **원문의 모든 세부 사항을 유지**하고 있음.
- 하지만 **문장이 짧음에도 불구하고 테이블이 상당히 큼**. $\rightarrow$ **차원축소** 과정이 필요.